In [7]:
from sympy import symbols, init_printing
import sympy
import sympy.physics.mechanics as me
init_printing(use_latex='mathjax')
import seaborn as sns
# sns.set_context("notebook", font_scale=1.5, rc={"lines.linewidth": 2.5})
import matplotlib.pyplot as plt
from scipy.integrate import ode, odeint
import numpy as np
# import InputShaping as shaping

# Create the variables
x, y, beta = me.dynamicsymbols('x, y, beta')

# Create the velocities
x_dot, y_dot, beta_dot = me.dynamicsymbols('x, y, beta', 1)

# Create the constants
m, k, L, g, H, c, D, t = sympy.symbols('m k L g H c D t')
Izz, k_beta, c_beta = sympy.symbols('Izz k_beta c_beta')
L_1_init, L_2_init = sympy.symbols('L_1_init L_2_init')
'''
m = mass
k = spring k
L = spring equilibrium length
g = gravity
c = spring c
c_beta = rotational c
k_beta = rotational k
D = rod length
Izz = moment of Inertia about the end of a rod
'''

####### Variables to Change ##############

# Beginning Point
X_begin = 14
Y_begin = 11.5

# Ending Point
X_end = 10
Y_end = 7.5

# Time to get to Point
Risetime = 3.0

# Time to start Moving
time_begin = 1.0

# Mass
mass = 1.0

# K of Cables
cable_K = 10.0

# C of Cables
cable_C = 1.0

# K of Rod
rod_K = 10.0

# C of Rod
rod_C = 1.0

# Length of Rod
rod_length = 3.0

# Time to plot response
endtime = 50.0
#########################################

inertia = rod_length**2 * (1.0/3.0) * mass

def Lengths(x,y):
    k=10
    h=20
    m=1.0
    Fy = np.array([[k*y/np.sqrt(x**2 + y**2), k*y/np.sqrt(y**2 + (h - x)**2)],
                   [k*x/np.sqrt(x**2 + y**2), -h*k/np.sqrt(y**2 + (h - x)**2
                    )+k*x/np.sqrt(y**2 + (h - x)**2)]])
    a = np.array([2*k*y - 9.81*m,-h*k + 2*k*x])
    x = np.linalg.solve(Fy,a)
    return x

L1_begin, L2_begin = Lengths(X_begin, Y_begin)
L1_end, L2_end = Lengths(X_end, Y_end)

In [8]:
# Create the world frame
N = me.ReferenceFrame('N')

# Create the rod frame
B = N.orientnew('B', 'axis', [beta, N.z])

# Set the rotation of the rod frame
B.set_ang_vel(N, -beta_dot * N.z)

# Create the Origin
O1 = me.Point('O_1')

# Set origin velocity to zero
O1.set_vel(N, 0 * N.x)

# Create the second attachment point
# O2 = O1.locatenew('O_2', H * N.x)
O2 = me.Point('O_2')
O2.set_pos(O1, H * N.x)
O2.set_vel(N, 0)

# Locate the point in the N frame
# P = me.Point('pen')
# P = O1.locatenew('P', x * N.x + y * N.y)
P = me.Point('P')
P.set_pos(O1, x * N.x + y * N.y)
P.set_pos(O2, -(H - x) * N.x + y * N.y)

# P.set_pos(O1, x * N.x + y * N.y)

# Set the point's velocity
P.set_vel(N, x_dot * N.x + y_dot * N.y)

# Create the rod center of mass
G = P.locatenew('G', D/2 * B.y)

# Set the velocity of G
G.v2pt_theory(P, N, B)

# Create the rod
I_rod = me.inertia(B, 0, 0, Izz)
rod = me.RigidBody('rod', G, B, m, (I_rod, G))

# Create the distance from the point to each attachment point
L1 = O1.pos_from(P).magnitude
L2 = O2.pos_from(P).magnitude
L1_vector = O1.pos_from(P).normalize
L2_vector = O2.pos_from(P).normalize

# Create the height from the center of gravity to the datum
h = G.pos_from(O1) & N.y

# The forces at the connection point
forceP = c * (x_dot + y_dot) * L1_vector() + c * (-x_dot + y_dot) * L2_vector()

# The forces on the beta frame
forceB = c_beta * beta_dot * N.z

rod.potential_energy = (-m * g * h + 0.5 * k * (L1() - L_1_init)**2 + 0.5 *
                        k *(L2() - L_2_init)**2 + 0.5 * k_beta * beta**2)

Lag = me.Lagrangian(N, rod)

LM = me.LagrangesMethod(Lag, [x, y, beta], forcelist=[(P, forceP), (B, forceB)], frame=N)


EqMotion = LM.form_lagranges_equations()

In [11]:
print(sympy.latex(Lag.subs({x_dot:'x_dot', x:'x',y:'y',y_dot:'y_dot', beta:'b', beta_dot:'beta_dot'})))

\frac{Izz \beta_{dot}^{2}}{2} - 0.5 b^{2} k_{\beta} + g m \left(\frac{D}{2} \cos{\left (b \right )} + y\right) - 0.5 k \left(- L_{1 init} + \sqrt{x^{2} + y^{2}}\right)^{2} - 0.5 k \left(- L_{2 init} + \sqrt{y^{2} + \left(H - x\right)^{2}}\right)^{2} + \frac{m}{2} \left(\frac{D^{2} \beta_{dot}^{2}}{4} + \frac{D \beta_{dot}}{2} x_{dot} \cos{\left (b \right )} + \frac{D \beta_{dot}}{2} y_{dot} \sin{\left (b \right )} + \frac{D \beta_{dot}}{2} \left(x_{dot} \cos{\left (b \right )} + y_{dot} \sin{\left (b \right )}\right) + x_{dot}^{2} + y_{dot}^{2}\right)


In [10]:
rod.potential_energy

                                                                  2           
                                   ⎛              _______________⎞          ⎛ 
      ⎛D⋅cos(β(t))       ⎞         ⎜             ╱  2       2    ⎟          ⎜ 
- g⋅m⋅⎜─────────── + y(t)⎟ + 0.5⋅k⋅⎝-L₁ ᵢₙᵢₜ + ╲╱  x (t) + y (t) ⎠  + 0.5⋅k⋅⎝-
      ⎝     2            ⎠                                                    

                                   2               
             _____________________⎞                
            ╱           2    2    ⎟            2   
L₂ ᵢₙᵢₜ + ╲╱  (H - x(t))  + y (t) ⎠  + 0.5⋅kᵦ⋅β (t)
                                                   

In [14]:
forceP

⎛             ⎛  d          d       ⎞     ⎛d          d       ⎞     ⎞
⎜c⋅(H - x(t))⋅⎜- ──(x(t)) + ──(y(t))⎟   c⋅⎜──(x(t)) + ──(y(t))⎟⋅x(t)⎟
⎜             ⎝  dt         dt      ⎠     ⎝dt         dt      ⎠     ⎟
⎜──────────────────────────────────── - ────────────────────────────⎟
⎜         _____________________                 _______________     ⎟
⎜        ╱           2    2                    ╱  2       2         ⎟
⎝      ╲╱  (H - x(t))  + y (t)               ╲╱  x (t) + y (t)      ⎠ n_x + ⎛    ⎛d          d       ⎞          ⎛  d          d       ⎞     ⎞
⎜  c⋅⎜──(x(t)) + ──(y(t))⎟⋅y(t)   c⋅⎜- ──(x(t)) + ──(y(t))⎟⋅y(t)⎟
⎜    ⎝dt         dt      ⎠          ⎝  dt         dt      ⎠     ⎟
⎜- ──────────────────────────── - ──────────────────────────────⎟
⎜          _______________              _____________________   ⎟
⎜         ╱  2       2                 ╱           2    2       ⎟
⎝       ╲╱  x (t) + y (t)            ╲╱  (H - x(t))  + y (t)    ⎠ n_y